# Lab 6 - Model comparison

In this excercise we will learn about comparing models using metrics predicting out of sample behavior.

Main idea is to instead of splitting the dataset into two parts (test and training set) to estimate how model would behave in presence of new data. This is being done by evaluating so called log likelihood which is an array of values of logarithm of likelihood for each of datapoints individually.

We will use this informations with two metrics:

- Watanabe-Akaike Information Criterion (also known as Widely Applicable Information Criterion, WAIC), which is averages log likelihood and estimates the effective number of paraemeters in the model
- PSIS-LOOCV - Pareto Smoothed Importance Sampling Leave-one-out Cross Validation. It is an estimate of value obtained from Leave-one-out Cross Validation by using modified importance sampling method instead of running inference N times where N is number of samples, leaving one each time.
For this excercise code is provided in form of print-screens.

In [11]:
from cmdstanpy import CmdStanModel
import arviz as az
import numpy as np
import scipy.stats as stats
import scipy.stats as norm
import matplotlib.pyplot as plt
import pandas as pd

## Excercise 1 - generate data

1. Compile code_1.stan and code_2.stan
2. Generate data for rest of excercises.

In [12]:
gen_quant = CmdStanModel(stan_file='code_1.stan')

F = 6           # F - number of letters in first name 
L = 6           # L - number of letters in last name 
N = (L+F)*100   # N = (L+F)*100

samples = gen_quant.sample(data={'N': F}, 
                            fixed_param=True,
                            iter_sampling=1000, 
                            iter_warmup=0, 
                            chains = 1)

# Creation of pandas dataframe from resulting draws
df = samples.draws_pd()
display(df)


INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


,lp__,accept_stat__,theta,y[1],y[2],y[3],y[4],y[5],y[6]
0,0.0,0.0,0.01,0.795930,-0.585080,1.093980,-0.325213,-0.814966,-7.024950
1,0.0,0.0,0.01,-2.179230,-0.652634,-1.544770,-1.637040,0.260985,-1.120530
2,0.0,0.0,0.01,-0.514749,0.699952,0.195412,1.034870,-0.314344,0.212859
3,0.0,0.0,0.01,0.624569,0.916228,-0.920655,-0.494420,0.256513,0.259929
4,0.0,0.0,0.01,-0.889798,-0.937814,-0.519742,-1.775170,1.451210,0.213932
...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,0.01,0.322214,0.703706,-1.003610,1.615510,0.129248,0.801144
996,0.0,0.0,0.01,-0.229033,-0.831032,1.237550,-0.789594,0.322308,1.245120
997,0.0,0.0,0.01,-1.890160,0.718657,-0.116001,1.808310,0.668286,0.216649
998,0.0,0.0,0.01,0.728326,-1.118010,-0.825295,0.260395,-1.129140,-0.135628


In [13]:
gen_quant = CmdStanModel(stan_file='code_2.stan')

F = 6           # F - number of letters in first name 
L = 6           # L - number of letters in last name 
N = (L+F)*100   # N = (L+F)*100

samples = gen_quant.sample(data={'N': F}, 
                            fixed_param=True,
                            iter_sampling=1000, 
                            iter_warmup=0, 
                            chains = 1)

# Creation of pandas dataframe from resulting draws
df = samples.draws_pd()
display(df)


INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


,lp__,accept_stat__,"X[1,1]","X[2,1]","X[3,1]","X[4,1]","X[5,1]","X[6,1]","X[1,2]","X[2,2]",...,"X[6,3]",beta[1],beta[2],beta[3],y[1],y[2],y[3],y[4],y[5],y[6]
0,0.0,0.0,-0.439087,1.066830,-1.615220,0.271533,0.608404,-0.230190,0.312941,0.272747,...,0.811741,2.0,1.0,0.5,-0.148305,1.58402,-4.783210,1.488510,0.045545,-0.509293
1,0.0,0.0,-0.341208,1.003040,-0.183340,0.560544,-0.485681,1.462540,0.614336,1.688890,...,1.512140,2.0,1.0,0.5,-2.213850,3.10670,-1.714790,1.117110,0.144174,2.595360
2,0.0,0.0,-0.349618,-1.020840,1.032280,0.723871,0.662351,1.853000,-0.000144,0.361129,...,-0.094874,2.0,1.0,0.5,-1.331230,-1.94173,3.459900,1.355240,1.738800,3.807530
3,0.0,0.0,-0.469300,0.933398,-1.083010,-0.036837,0.229178,2.209160,-1.382850,0.570664,...,1.156890,2.0,1.0,0.5,-3.229610,2.71097,-2.400480,-2.177610,0.025464,4.267280
4,0.0,0.0,-0.175628,-0.912035,0.010404,0.012205,1.014300,-0.573805,1.598840,-0.219008,...,0.086114,2.0,1.0,0.5,1.456720,-4.25963,0.850465,0.485215,1.710640,-1.846060
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.0,0.0,1.141770,-0.072106,0.325648,-1.332610,0.612552,-1.299110,1.950370,-1.171800,...,0.728212,2.0,1.0,0.5,5.285590,-0.67553,-0.760305,-3.304270,0.968393,-2.462900
996,0.0,0.0,0.892786,0.569862,-1.309720,-0.631858,0.623860,1.509590,-2.012530,-0.511828,...,-1.491150,2.0,1.0,0.5,-0.413620,0.52539,-0.385074,-1.958700,0.045807,1.989470
997,0.0,0.0,-0.619184,1.712500,0.007727,1.344740,-0.948403,-0.077655,0.160762,-0.340423,...,0.432176,2.0,1.0,0.5,-0.562610,3.19739,0.447318,2.723830,-0.448076,2.237520
998,0.0,0.0,-0.474474,-0.999975,-0.509370,0.160636,-1.590710,-1.063660,0.087340,-0.882691,...,1.645250,2.0,1.0,0.5,0.598322,-3.18230,-0.350084,0.281524,-4.155170,-2.248360


## Excercise 2 - compare normal and student models for data from first file

1. Compile both models
2. Fit both models
3. Using az.compare and az.plot_compare analyze both models using ```loo``` and ```waic``` criteria.

In [14]:
# gen_quant2 = CmdStanModel(stan_file='code_3.stan')

# F = 6           # F - number of letters in first name 
# L = 6           # L - number of letters in last name 
# N = (L+F)*100   # N = (L+F)*100

# # samples = gen_quant.sample(data={'N': F}, 
# #                             fixed_param=True,
# #                             iter_sampling=1000, 
# #                             iter_warmup=0, 
# #                             chains = 1)

# # Creation of pandas dataframe from resulting draws
# df = samples.draws_pd()
# display(df)

In [17]:
gen_data = CmdStanModel(stan_file='code_4.stan')

F = 6           # F - number of letters in first name 
L = 6           # L - number of letters in last name 
N = (L+F)*100   # N = (L+F)*100

# samples = gen_data.sample(data={'N': F}, 
#                             fixed_param=True,
#                             iter_sampling=1000, 
#                             iter_warmup=0, 
#                             chains = 1)

sim = gen_data.sample(iter_sampling=N,
                     iter_warmup=0,
                     chains=1,
                     fixed_param=True)

# # Creation of pandas dataframe from resulting draws
# data = samples.draws_pd()
# display(data)

INFO:cmdstanpy:found newer exe file, not recompiling
INFO:cmdstanpy:CmdStan start processing
chain 1 |          | 00:00 StatusERROR:cmdstanpy:Chain [1] error: error during processing Operation not permitted
chain 1 |██████████| 00:00 Sampling completed


INFO:cmdstanpy:CmdStan done processing.


RuntimeError: Error during sampling:
Exception: variable does not exist; processing stage=data initialization; variable name=N; base type=int (in '/home/LaboratoryClasses_Data_Analitycs/Lab_6_Model_comparison/code_4.stan', line 2, column 4 to column 10)Command and output files:
RunSet: chains=1, chain_ids=[1], num_processes=1
 cmd (chain 1):
	['/home/LaboratoryClasses_Data_Analitycs/Lab_6_Model_comparison/code_4', 'id=1', 'random', 'seed=58630', 'output', 'file=/tmp/tmpcpohl0x_/code_4-20230417101233.csv', 'method=sample', 'num_samples=1200', 'num_warmup=0', 'algorithm=fixed_param']
 retcodes=[1]
 per-chain output files (showing chain 1 only):
 csv_file:
	/tmp/tmpcpohl0x_/code_4-20230417101233.csv
 console_msgs (if any):
	/tmp/tmpcpohl0x_/code_4-20230417101233_0-stdout.txt

## Excercise 3 - compare models with different numbers of predictors

1. Compile model
2. Compare models for 1, 2 and 3 predictors as in previous excercise